# Queensland Car Crash Data Analysis and Visualisation

[Crash data from Queensland roads](https://www.data.qld.gov.au/dataset/crash-data-from-queensland-roads) contains data about road crash locations, road casualties, driver demographics, seatbelt restraits and helmet use, vehicle types and factors in road crashes. 

The data contains information on location and characteristics of crashes in Queensland for all reported Road Traffic Crashes occurred from 1 January 2001 to 31 December 2020, Fatal Road Traffic Crashes to 31 December 2020, Hospitalisation, Medical Treatment and Minor Injury Crashes to 31 December 2020 and Property Damage only crashes to 31 December 2010.

**Summary of datasets:**
- Road crash locations: Location and characteristics of crashes within Queensland for all reported Road Traffic Crashes
- Road casualties: Characteristics of casualties as a result of crashes within Queensland for all reported Road Traffic Crashes
- Driver demographics: Driver involvement in crashes within Queensland for all reported Road Traffic Crashes
- Seatbelt restraints and helmet use: Restraint use of vehicle occupant casualties, and helmet use of motorcycle riders/pillion casualties and bicycle rider/pillion casualties as a result of crashes within Queensland for all reported Road Traffic Crashes
- Vehicle types: Vehicle involvement in crashes within Queensland for all reported Road Traffic Crashes
- Factors in road crashes: Alcohol, speed, fatigue and defective vehicle involvement in crashes within Queensland for all reported Road Traffic Crashes

**Key takeaways**

From 2001 to 2020:
- The road with the most crashes was Bruce Highway with 12300 crashes followed by Pacific Highway with 5169 crashes.
- The road with the most fatalities was also Bruce Highway with 719 total fatalities, followed by Warrego Highway with 120.
- The day of the week with the most crashes was on Friday, and the day with the least was Sunday.
    - On Friday, there is a 45.86% higher chance of being in a car crash than on Sunday
- The time of day with the most crashes was 3pm, followed by 4pm, 5pm and 8 am. 
- The most common crash severity was medical treatment, followed by hospitalisation, minor injury and then fatality
- Almost 45% of fatal crashes occurred on roads with a 100-110 km/h speed limit
- Over 50% of crashes occur on roads with a 60 km/h speed limit 
- The age group that occurs most in crashes are 17-24 years of age followed by 30-39

---
## Road Crash Locations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math

import altair as alt
alt.renderers.enable('html')

RendererRegistry.enable('html')

In [2]:
# load road crash locations data
# rcl = pd.read_csv('1_crash_locations.csv')

# # using url
rcl_url = 'https://www.data.qld.gov.au/dataset/f3e0ca94-2d7b-44ee-abef-d6b06e9b0729/resource/e88943c0-5968-4972-a15f-38e120d72ec0/download/1_crash_locations.csv'
rcl = pd.read_csv(rcl_url)

In [3]:
rcl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353651 entries, 0 to 353650
Data columns (total 52 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Crash_Ref_Number                 353651 non-null  int64  
 1   Crash_Severity                   353651 non-null  object 
 2   Crash_Year                       353651 non-null  int64  
 3   Crash_Month                      353651 non-null  object 
 4   Crash_Day_Of_Week                353651 non-null  object 
 5   Crash_Hour                       353651 non-null  int64  
 6   Crash_Nature                     353651 non-null  object 
 7   Crash_Type                       353651 non-null  object 
 8   Crash_Longitude_GDA94            353651 non-null  float64
 9   Crash_Latitude_GDA94             353651 non-null  float64
 10  Crash_Street                     353636 non-null  object 
 11  Crash_Street_Intersecting        153069 non-null  object 
 12  St

In [4]:
rcl.head()

,Crash_Ref_Number,Crash_Severity,Crash_Year,Crash_Month,Crash_Day_Of_Week,Crash_Hour,Crash_Nature,Crash_Type,Crash_Longitude_GDA94,Crash_Latitude_GDA94,...,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_Total,Count_Unit_Car,Count_Unit_Motorcycle_Moped,Count_Unit_Truck,Count_Unit_Bus,Count_Unit_Bicycle,Count_Unit_Pedestrian,Count_Unit_Other
0,1,Property damage only,2001,January,Tuesday,16,Overturned,Single Vehicle,0.000000,0.000000,...,0,0,0,1,0,0,0,0,0,0
1,2,Property damage only,2001,January,Wednesday,17,Hit object,Single Vehicle,153.060732,-27.587883,...,0,0,0,1,0,0,0,0,0,0
2,3,Medical treatment,2001,January,Thursday,9,Hit pedestrian,Hit pedestrian,153.016436,-27.560422,...,1,0,1,0,0,1,0,0,1,0
3,4,Hospitalisation,2001,February,Thursday,7,Rear-end,Multi-Vehicle,153.049047,-27.598150,...,0,0,1,2,0,0,0,0,0,0
4,5,Property damage only,2001,February,Thursday,11,Hit object,Single Vehicle,152.998562,-27.564987,...,0,0,0,0,0,1,0,0,0,0


In [5]:
rcl['Count_Casualty_Total'].describe()

count    353651.000000
mean          1.004541
std           0.870901
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          32.000000
Name: Count_Casualty_Total, dtype: float64

In [6]:
rcl.shape

(353651, 52)

There are 353651 crash reports in the period from 1 January 2001 to 31 December 2020.

In [7]:
# 2020 only data
rcl_2020 = rcl[rcl['Crash_Year'] == 2020]
rcl_2020.shape

(11937, 52)

### Visualising the data on a map

In [8]:
# https://github.com/python-visualization/folium/tree/master/examples

# import sys
# !{sys.executable} -m pip install folium

import folium
from folium import Marker
from folium.plugins import MarkerCluster, FastMarkerCluster, ScrollZoomToggler, Fullscreen

def make_map(map_data):
    
    brisbane = [-27.467778, 153.028056]
    m = folium.Map(location=brisbane,
               zoom_start=7,
               tiles="CartoDB positron")
    
    # add stuff
    Fullscreen(position="topright", force_separate_button=True).add_to(m)

    # add marker cluster
    mc = MarkerCluster()
    # TODO: try use FastMarkerCluster
    
    for idx, row in map_data.iterrows():
        if not np.isnan(row.Crash_Latitude_GDA94) and \
            not np.isnan(row.Crash_Longitude_GDA94):
            popuptext = f"Crash time: Hour = {row.Crash_Hour}, {row.Crash_Day_Of_Week} {row.Crash_Month} {row.Crash_Year}<br>\
                    Crash Severity: {row.Crash_Severity}<br>\
                    Crash Type: {row.Crash_Type}<br>\
                    Crash Street Location: {row.Crash_Street} {row.Loc_Suburb} {row.Loc_Post_Code}<br>\
                    Crash Street Intersecting: {row.Crash_Street_Intersecting if not math.nan else 'None'}<br>\
                    Local Government Area: {row.Loc_Local_Government_Area}<br>\
                    Speed Limit: {row.Crash_Speed_Limit}<br>\
                    Casualty Total: {row.Count_Casualty_Total}<br>\
                    Fatality Total: {row.Count_Casualty_Fatality}<br>\
                    Crash Ref Number: {row.Crash_Ref_Number}"
            popup = folium.Popup(popuptext, min_width=300, max_width=500)
            if row.Crash_Severity == 'Fatal':
                icon_color = 'red'
            elif row.Crash_Severity == 'Hospitalisation' or row.Crash_Severity == 'Minor injury':
                icon_color = 'orange'
            else:
                icon_color = 'beige'
            mc.add_child(Marker([row.Crash_Latitude_GDA94, row.Crash_Longitude_GDA94], 
                                popup=popup,
                                icon=folium.Icon(color=icon_color)
                               )
                        )
    
    # adds marker cluster to map
    m.add_child(mc)
    
    return m

In [9]:
# make_map(rcl_2020)

In [10]:
street = 'Brisbane St'
street_data = rcl[rcl['Crash_Street'] == street]
# make_map(street_data)

In [11]:
def subset_data(data, year=None, suburb=None, street=None):
    if year:
        print("year")
        data = data[data['Crash_Year'] == year]

    if suburb:
        try:
            print("suburb")

            data = data[data['Loc_Suburb'] == suburb]
        except:
            print(f"{suburb} not found")

    if street:
        try:
            print("Street")
            data = data[data['Crash_Street'] == street]
        except:
            print(f"{street} not found")
    return data

### Total crash count each year

In [12]:
total_year = rcl.groupby('Crash_Year').size().reset_index(name='Count')
alt.Chart(total_year).mark_bar().encode(
    x='Crash_Year:O',  # https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types
    y='Count'
).properties(title='Total crash count each year')

alt.Chart(...)

There is a big drop in crash count from 2010 to 2011. I wondered why that is and then looked at the website again and found that *"Property damage only crashes ceased to be reported/recorded by Queensland Police Service after 31 December 2010"* which explains the big drop. 

Since I'm only interested in finding how dangerous a road is I will ignore property damage only crashes. Dangerous meaning roads that have a high crash, casualty or fatality count. 

In [13]:
rcl = rcl[rcl.Crash_Severity != 'Property damage only']

total_year2 = rcl.groupby('Crash_Year').size().reset_index(name='Count')
alt.Chart(total_year2).mark_bar().encode(
    x='Crash_Year:O',  # https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types
    y='Count'
).properties(title='Total crash count each year (excl. property damage)')

alt.Chart(...)

We can see that the total crash count each year (excluding property damage only crashes) has decreased a bit from 2001 to 2020. 

Let's graph percentage change to see how the crash count has changed each year:

In [14]:
total_year2['pct_change'] = total_year2.Count.pct_change()

alt.Chart(total_year2).mark_bar().encode(
    x='Crash_Year:O',  # https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types
    y=alt.Y('pct_change', axis=alt.Axis(format='%', title='% change')),
    color=alt.condition(
        alt.datum.pct_change > 0,
        alt.value("orangered"),
        alt.value("steelblue")
    )
).properties(title='Percentage change in total crash count each year')

alt.Chart(...)

It looks like 2020 has seen the biggest decrease in crash count, which may be due to less drivers on the road because of COVID and lockdowns. 

It's also interesting to see that from 2009 to 2011, there has been a steady decrease in crash count of at least 4% each year. 

Another possible graph could be to get pct of crashes each year out of total population each year. 

### Road with most crashes

In [15]:
roads = rcl.groupby('Crash_Street').size().reset_index(name='Count')
roads = roads.sort_values(by='Count', ascending=False)

alt.Chart(roads.head(10)).mark_bar().encode(
    x='Count',
    y=alt.Y('Crash_Street:O', sort='-x')  # sorted https://altair-viz.github.io/gallery/bar_chart_sorted.html
).properties(title=f"Total crash count on each road")

alt.Chart(...)

Bruce Highway has the largest total car crash count. This may be because Bruce Highway is 1670 km long. The next road with most crashes is the Pacific Highway which is 919 km long.

This may mean that Bruce Highway may not be the most dangerous road because part of the reason for the high crash count can be attributed to the fact that it's the longest road in the data. 

One way to compare the road's crash likelihood could be to calculate the crash count per km for each road. I have not been able to find a data set that gives length of reach road/street so I probably won't be able to do this.

Since my goal is to figure out which roads are most dangeorus i.e. has the most casualties, I will look at the casualty statistics for each road. 

Source: http://www.bonzle.com/c/a?a=f&sc=lr&st=3&cmd=sp

In [16]:
top_road = roads.iloc[0].Crash_Street
top_road_data = rcl[rcl.Crash_Street == top_road]
top_road_yearly = top_road_data.groupby('Crash_Year').size().reset_index(name='Count')

alt.Chart(top_road_yearly).mark_bar().encode(
    x='Crash_Year:O',
    y='Count').properties(
    title=f"Total crash count each year on {top_road}"
)

alt.Chart(...)

In [17]:
top_road_yearly = top_road_data.groupby('Crash_Year').size().reset_index(name='Count')
top_road_yearly['pct_change'] = top_road_yearly.Count.pct_change()

alt.Chart(top_road_yearly).mark_bar().encode(
    x='Crash_Year:O',
    y=alt.Y('pct_change', axis=alt.Axis(format='%', title='% change'))
).properties(title=f"Percentage change in total crash count each year on {top_road}")

alt.Chart(...)

In [18]:
# plotting as proportion of total car crashes instead for each year
road_year = rcl.groupby('Crash_Year').size().reset_index(name='Count')
top_road_yearly['prop'] = top_road_yearly.Count / road_year.Count

alt.Chart(top_road_yearly).mark_bar().encode(
    x='Crash_Year:O',
    y=alt.Y('prop', axis=alt.Axis(format='%', title='Proportion'))
).properties(title=f"Proportion of crashes {top_road} each year out of total crashes each year")

alt.Chart(...)

In [19]:
# top_road_data.groupby('Crash_Year')['Crash_Severity'].value_counts()

In [20]:
top_road_casualty_total = top_road_data.groupby('Crash_Year').Count_Casualty_Total.sum().reset_index(name='Count')

alt.Chart(top_road_casualty_total).mark_bar().encode(
    x='Crash_Year:O',
    y='Count').properties(
    title=f"Total casualty count each year on {top_road}"
)

alt.Chart(...)

In [21]:
top_road_fatal_total = top_road_data.groupby('Crash_Year').Count_Casualty_Fatality.sum().reset_index(name='Count')

alt.Chart(top_road_fatal_total).mark_bar().encode(
    x='Crash_Year:O',
    y='Count').properties(
    title=f"Total fatality count each year on {top_road}"
)

alt.Chart(...)

The casualty count on Bruce Highway seems to be decreasing over the years. There does not, however, seem to be a pattern to fatality count each year.

### Roads with most casualties

In [22]:
# NOTE: casualty is always >= 1 if not property damage
casualty_road_data = (rcl.groupby('Crash_Street')['Count_Casualty_Total']
                      .sum().reset_index(name='Count')
                      .sort_values(by='Count', ascending=False)
                     )
alt.Chart(casualty_road_data.head(10)).mark_bar().encode(
    x='Count',
    y=alt.Y('Crash_Street:O', sort='-x')  # sorted https://altair-viz.github.io/gallery/bar_chart_sorted.html
).properties(title=f"Total casualty count on each road")

alt.Chart(...)

### Roads with most fatalities

In [23]:
fatal_road_data = (rcl.groupby('Crash_Street')['Count_Casualty_Fatality']
                      .sum().reset_index(name='Count')
                      .sort_values(by='Count', ascending=False)
                     )
alt.Chart(fatal_road_data.head(10)).mark_bar().encode(
    x='Count',
    y=alt.Y('Crash_Street:O', sort='-x')  # sorted https://altair-viz.github.io/gallery/bar_chart_sorted.html
).properties(title=f"Total fatality count on each road")

alt.Chart(...)

It is interesting to note that the ranking of crash count, casualty count and fatality count are not the same. For instance, Bruce, Pacific and Gold Coast Highway are first, second and third in total crash count and total casualty count but in the fatality count, Warrego Highway is second and Cunningham Highway is third while Bruce Highway remains first in fatality count. 

This suggests that another way to measure how dangerous a road is by counting the total number of fatalities out of the total number of crashes or the total number of casualties?

### Fatality count per crash for each road

In [24]:
fatal_count_street = (rcl.groupby('Crash_Street')['Count_Casualty_Fatality']
                      .sum().reset_index(name='fatal_count')
                      .sort_values(by='fatal_count', ascending=False)
                     )

fatal_count_street.head()

,Crash_Street,fatal_count
2924,Bruce Hwy,719
18262,Warrego Hwy,120
5207,Cunningham Hwy,75
14257,Pacific Hwy,71
13549,New England Hwy,67


In [25]:
roads_fatal_count = pd.merge(roads, fatal_count_street)
roads_fatal_count['pct'] = roads_fatal_count['fatal_count'] / roads_fatal_count['fatal_count'].sum()

roads_fatal_count.sort_values(by='pct', ascending=False, inplace=True)

c = alt.Chart(roads_fatal_count.head(10)).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Fatality count per crash')),
    y=alt.Y('Crash_Street:O', title='Casualty Age Group', sort='-x') 
).properties(title=f"Fatal count per crash for each road")


# https://altair-viz.github.io/gallery/bar_chart_with_labels.html

c_labels = c.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=alt.Text('pct', format='.2%')
)

c + c_labels

alt.LayerChart(...)

The Warrego Highway has a higher fatality count per crash than Bruce Highway while Bruce Highway has higher total crash and fatality count. This indicates that there are 0.0682 fatalities per crash on Warrego Highway or that there is 1 fatality for every 1/0.0682 = 14 crashes?

### Roads with most crashes/fatalities given a suburb

In [26]:
suburb = 'St Lucia'
suburb_data = rcl[rcl['Loc_Suburb'] == suburb]
# make_map(suburb_data)

In [27]:
def graph_crash_street(data):
    suburb_name = data.iloc[0].Loc_Suburb
    streets = (data.groupby('Crash_Street')
               .size()
               .reset_index(name='Count')
               .sort_values(by='Count', ascending=False)
              )
    
    chart = alt.Chart(streets.head(10)).mark_bar().encode(
        x='Count',
        y=alt.Y('Crash_Street:O', sort='-x')  # sorted https://altair-viz.github.io/gallery/bar_chart_sorted.html
    )
    chart_text = chart.mark_text(
        align='left',
        baseline='middle',
        dx=3
    ).encode(
        text='Count'
    )
    
    return (chart + chart_text).properties(title=f"Streets with the most crashes on {suburb_name}")

In [28]:
graph_crash_street(suburb_data)

alt.LayerChart(...)

### Frequency of crashes per day of the week 

In [29]:
day_of_weeks = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

crash_per_day = rcl.groupby('Crash_Day_Of_Week').size().reset_index(name='Count')

crash_per_day['pct'] = crash_per_day.Count / crash_per_day.Count.sum()

alt.Chart(crash_per_day).mark_bar().encode(
    x=alt.X('Crash_Day_Of_Week', sort=day_of_weeks),
    y=alt.Y('pct', axis=alt.Axis(format='%', title='Proportion'))
).properties(title=f"Frequency of crashes per day of the week ")

alt.Chart(...)

In [30]:
alt.Chart(crash_per_day).mark_bar().encode(
    x=alt.X('Crash_Day_Of_Week', sort=day_of_weeks),
    y = alt.Y('pct', axis=alt.Axis(format='%', title='Proportion'))
).properties(title=f"Relative frequency of crashes per day of the week")

alt.Chart(...)

From 2001 to 2020, the day of the week with the lowest frequency of crashes is Sunday, and it increases for each day and reaches a peak on Friday. I believe the peak on Friday can be explained by the fact that most people go out on Friday and the minimum on Sunday is because most people are home earlier to get ready for work on the following Monday. Another possible thing to do is get traffic data and see how much cars there are on the road at those days of the weeks.

How much more likely are you to be in a crash on Friday than on Sunday? Since the relative frequency on Friday is 16.57% and on Sunday it is 11.36%, we can say that on Friday, you are $\frac{16.57-11.36}{11.36} = 0.4586 = 45.85\%$ more likely to be in a car crash than on Sunday?

In [31]:
# crash_per_day_2020 = rcl_2020.groupby('Crash_Day_Of_Week').size().reset_index(name='Count')
# crash_per_day_2020['pct'] = crash_per_day_2020.Count / crash_per_day_2020.Count.sum()

# alt.Chart(crash_per_day_2020).mark_bar().encode(
#     x=alt.X('Crash_Day_Of_Week', sort=day_of_weeks),
#     y='Count'
# ).properties(title=f"Total number of crashes per day of the week in 2020")
# # TODO: per year

In [32]:
# alt.Chart(crash_per_day_2020).mark_bar().encode(
#     x=alt.X('Crash_Day_Of_Week', sort=day_of_weeks),
#     y=alt.Y('pct', axis=alt.Axis(format='%', title='Proportion'))
# ).properties(title=f"Proportion of crashes per day of the week in 2020")

### Crashes per hour of the day

In [33]:
# crash hour
crash_per_hour = rcl.groupby('Crash_Hour').size().reset_index(name='Count')
crash_per_hour['pct'] = crash_per_hour.Count / crash_per_hour.Count.sum()

# alt.Chart(crash_per_hour).mark_bar().encode(
#     x='Crash_Hour:O',
#     y='Count'
# ).properties(title='Total crash count per hour of the day')

alt.Chart(crash_per_hour).mark_bar().encode(
    x='Crash_Hour:O',
    y=alt.Y('pct', axis=alt.Axis(format='%', title='Proportion'))
).properties(title=f"Proportion of crashes per hour of the day")

alt.Chart(...)

In [34]:
# crash_per_hour['Total_count'] = rcl.groupby('Crash_Hour').size()

# alt.Chart(crash_per_hour).transform_fold(
#     ['Count', 'Total_count']
# ).mark_bar(opacity=0.7).encode(
#         x='Crash_Hour:O',
#         y=alt.Y('value:Q'),  # can't get stack=None to work when using transform_fold?
#         color=alt.Color('key:N', title='source')
#     )

I believe the increase in the 8th hour may be due to increase in rush hour traffic to go to work. And similarly, in the afternoon hours of 2-5pm we see increase in crashes, likely as a result of rush hour going home from work. 

We have the fewest crashes in the early morning around 2-4am.

In [35]:
crash_per_day_hour = rcl.groupby(['Crash_Day_Of_Week', 'Crash_Hour']).size().reset_index(name='Count')
crash_per_day_hour.head()

friday_vs_sunday = crash_per_day_hour[(crash_per_day_hour.Crash_Day_Of_Week == 'Friday') | 
                                      (crash_per_day_hour.Crash_Day_Of_Week == 'Sunday')]

alt.Chart(friday_vs_sunday).mark_bar(opacity=0.7).encode(
    x='Crash_Hour:O',
    y=alt.Y('Count', stack=None),  # stack = none prevents stacking
    color='Crash_Day_Of_Week'
).properties(title='Crash count per hour of the day in 2020 for Friday and Sunday')

alt.Chart(...)

In [36]:
alt.Chart(crash_per_day_hour).mark_bar().encode(
    x='Crash_Hour:O',
    y=alt.Y('Count', stack=None),
    color='Crash_Day_Of_Week'
).facet(   
    column='Crash_Day_Of_Week'
).properties(title='Crash count per hour of day per week day')

alt.FacetChart(...)

We can see that the morning and afternoon peaks we see on the week days disappear on the weekends

### Total number of crashes per month

In [37]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September' 'October', 'November', 'December']
crash_per_month = rcl.groupby('Crash_Month').size().reset_index(name='Count')
crash_per_month['pct'] = crash_per_month.Count / crash_per_month.Count.sum()

# TODO: why is it not ordered by months?
alt.Chart(crash_per_month).mark_bar().encode(
    x=alt.X('Crash_Month', sort=months),
    y=alt.Y('pct', axis=alt.Axis(format='%', title='Proportion'))
).properties(title=f"Proportion of crashes per month")

alt.Chart(...)

### Summary of road crash location data

The following is just getting value counts and their proportion

In [38]:
crash_severity = rcl['Crash_Severity'].value_counts(normalize=True).reset_index(name='pct')

alt.Chart(crash_severity).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Crash Severity', sort='-x')
).properties(title="Crash Severity percentage")

alt.Chart(...)

In [39]:
crash_nature = rcl['Crash_Nature'].value_counts(normalize=True).reset_index(name='pct')

alt.Chart(crash_nature).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Crash Nature', sort='-x')
).properties(title="Crash Nature percentage")

alt.Chart(...)

In [40]:
crash_type = rcl['Crash_Type'].value_counts(normalize=True).reset_index(name='pct')

alt.Chart(crash_type).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Crash Type', sort='-x')
).properties(title="Crash Type percentage")

alt.Chart(...)

In [41]:
roadway_feature = (rcl['Crash_Roadway_Feature'].value_counts(normalize=True)
                   .reset_index().rename(
                       columns={'index':'Crash_Roadway_Feature', 
                                'Crash_Roadway_Feature': 'pct'})
                   ) # there must be a better way to do this

alt.Chart(roadway_feature).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Crash_Roadway_Feature:O', sort='-x')
).properties(title=f"Total roadway feature count as percentage")

alt.Chart(...)

In [42]:
speed_limit = rcl.Crash_Speed_Limit.value_counts(normalize=True).reset_index(name='pct')

alt.Chart(speed_limit).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Speed limit', sort='-x')
).properties(title=f"Speed limit percentage for each crash")

alt.Chart(...)

Over 50% of crashes occur when the speed limit is 60 km/h. 

In [43]:
road_surface = rcl.Crash_Road_Surface_Condition.value_counts(normalize=True).reset_index(name='pct')

alt.Chart(road_surface).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Road Surface Condition', sort='-x') 
).properties(title=f"Road surface condition percentage for each crash")

alt.Chart(...)

In [44]:
atmosphere = rcl.Crash_Atmospheric_Condition.value_counts(normalize=True).reset_index(name='pct')

alt.Chart(atmosphere).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Atmospheric condition', sort='-x') 
).properties(title=f"Atmospheric condition percentage for each crash")

alt.Chart(...)

Almost 90% of crashes occur in clear weather and just over 10% occur when it is raining. 

In [45]:
lighting = rcl.Crash_Lighting_Condition.value_counts(normalize=True).reset_index(name='pct')

alt.Chart(lighting).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Lighting condition', sort='-x') 
).properties(title=f"Lighting condition percentage for each crash")

alt.Chart(...)

In [46]:
description = rcl.Crash_DCA_Group_Description.value_counts(normalize=True).reset_index(name='pct')
# rcl.Crash_DCA_Description.value_counts(normalize=True).head(10) # similar 

alt.Chart(description).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='DCA Group Description', sort='-x') 
).properties(title=f"DCA Group Description percentages")

alt.Chart(...)

Most crashes are rear ends! That's pretty good to hear. However, followed closely is Opposite approach which is bad. 3.2% of crashes are head on!

In [47]:
approach_dir = rcl.DCA_Key_Approach_Dir.value_counts(normalize=True).reset_index(name='pct')

alt.Chart(approach_dir).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='DCA Key Approach Direction', sort='-x') 
).properties(title=f"DCA Key Approach Direction percentages")

alt.Chart(...)

### Crash Severity at 60 km/h

In [48]:
sixtykmh = (rcl[rcl['Crash_Speed_Limit'] == '60 km/h']['Crash_Severity']
            .value_counts(normalize=True)
            .reset_index(name='Count')
           )

alt.Chart(sixtykmh).mark_bar().encode(
    x=alt.X('Count', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Crash Severity', sort='-x') 
).properties(title=f"Crash Severity percentages on 60 km/h speed limit roads")

alt.Chart(...)

### Crash Severity at 100-110 km/h

In [49]:
hundredkmh = (rcl[rcl['Crash_Speed_Limit'] == '100 - 110 km/h']['Crash_Severity']
            .value_counts(normalize=True)
            .reset_index(name='Count')
           )

alt.Chart(hundredkmh).mark_bar().encode(
    x=alt.X('Count', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Crash Severity', sort='-x') 
).properties(title=f"Crash Severity percentages on 100-110 km/h speed limit roads")

alt.Chart(...)

It looks like there are more fatal crashes on 100-110 km/h roads than on 60 km/h roads. Just over 5% of crashes on 100-110 km/h speed limit roads are fatal whereas less than 1% of crashes on 60 km/h speed limit roads are fatal.

In [50]:
speed_fatal = (rcl[rcl['Crash_Severity'] == 'Fatal']
               .groupby(['Crash_Speed_Limit', 'Crash_Severity'])
               .size().reset_index(name='Count')
              )

speed_fatal['pct'] = speed_fatal['Count'] / speed_fatal['Count'].sum()

speed_fatal_chart = alt.Chart(speed_fatal).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Crash_Speed_Limit:O', sort='-x') 
)

speed_fatal_text = speed_fatal_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=alt.Text('pct', format='.2%')
)

(speed_fatal_chart + speed_fatal_text).properties(title=f"Percentage of crashes that are fatal for each speed limit")

alt.LayerChart(...)

The above graph shows that out of all fatal crashes, almost 45% of fatal were in roads with 100 - 110 km/h speed limits! 

In [51]:
speed_hospitalisation = (rcl[rcl['Crash_Severity'] == 'Hospitalisation']
               .groupby(['Crash_Speed_Limit', 'Crash_Severity'])
               .size().reset_index(name='Count')
              )

speed_hospitalisation['pct'] = speed_hospitalisation['Count'] / speed_hospitalisation['Count'].sum()

speed_hospitalisation_chart = alt.Chart(speed_hospitalisation).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Crash_Speed_Limit:O', sort='-x') 
)

speed_hospitalisation_text = speed_hospitalisation_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=alt.Text('pct', format='.2%')
)

(speed_hospitalisation_chart + speed_hospitalisation_text).properties(title=f"Percentage of crashes that result in hospitalisation for each speed limit")

alt.LayerChart(...)

In [52]:
speed_severity = rcl[['Crash_Speed_Limit', 'Crash_Severity']].value_counts(normalize=True).reset_index(name='pct')  #normalize seems to return relative frequency out of all groups instead of each speed limit subgroup
speed_severity

speed_sort = ['0 - 50 km/h', '60 km/h', '70 km/h', '80 - 90 km/h', '100 - 110 km/h']
severity_sort = ['Medical treatment', 'Minor injury', 'Hospitalisation', 'Fatal']

alt.Chart(speed_severity).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage'), stack="normalize"),  # normalized https://altair-viz.github.io/gallery/normalized_stacked_bar_chart.html
    y=alt.Y('Crash_Severity:O', sort=severity_sort),
    color='Crash_Speed_Limit'
).properties(title='Crash Severity proportion for each severity and speed limit category')

alt.Chart(...)

In [53]:
alt.Chart(speed_severity).mark_bar().encode(
    x=alt.X('Crash_Severity:O', sort=severity_sort),
    y=alt.Y('pct', axis=alt.Axis(format='%', title='Percentage')),  # normalized https://altair-viz.github.io/gallery/normalized_stacked_bar_chart.html
    color='Crash_Severity', 
    column=alt.Column('Crash_Speed_Limit', sort=speed_sort)
).properties(title='Crash Severity proportion for each severity and speed limit category')

# change colour so fatal is red etc

alt.Chart(...)

---
## Road Casualties

The Road Crash Locations includes some data on casualty type, this data set gives more details about the casualty such as age group gender and the road user type

In [54]:
# road casualties
# rc = pd.read_csv('a_road_casualties.csv'

rc = pd.read_csv('https://www.data.qld.gov.au/dataset/f3e0ca94-2d7b-44ee-abef-d6b06e9b0729/resource/3fc53539-d529-4c1d-85f8-6c92d9e06fc8/download/a_road_casualties.csv')

<!-- There are some large counts in the `Casualty_Count` column which seems to suggest that the data is grouped together. Furthermore, the number of rows is much smaller than in the road crash locations data set. Because the casualty count is not the same as the number of rows this means that to calculate the relative frequencies (`value_counts()`) we would need to sum the `Casualty_Count` for each group first. 
 -->
### Casualty Severity

In [55]:
rc_severity = rc.groupby('Casualty_Severity')['Casualty_Count'].sum().reset_index(name='count')
rc_severity['pct'] = rc_severity['count'] / rc_severity['count'].sum()

alt.Chart(rc_severity).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Casualty_Severity:O', title='Casualty Severity', sort='-x') 
).properties(title=f"Casualty Severity percentages")

alt.Chart(...)

NOTE: The graph above gives a significantly different graph to the one below where I just use value_counts which just uses the number of rows for each age group. For instance, the fatality rate in the graph below is way too high at over 10% whereas it's a more accurate < 5 % in the above graph.

In [56]:
rc_severity2 = rc.Casualty_Severity.value_counts(normalize=True).reset_index(name='pct')

alt.Chart(rc_severity2).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Casualty Severity', sort='-x') 
).properties(title=f"Casualty Severity percentages (bad)")

alt.Chart(...)

In [57]:
alt.Chart(crash_severity).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('index:O', title='Crash Severity', sort='-x')
).properties(title="Casualty Severity percentage (from road crash locations data)")

alt.Chart(...)

The first graph above looks more similar to the one from the road crash locations dataset than the second one which means its more accurate to take the sum of the `Casualty_Count` column to make the graphs

### Casualty Age Group

In [58]:
rc_casualty_age = rc.groupby('Casualty_AgeGroup')['Casualty_Count'].sum().reset_index(name='count') # .value_counts(normalize=True).reset_index(name='pct')
rc_casualty_age['pct'] = rc_casualty_age['count'] / rc_casualty_age['count'].sum()

alt.Chart(rc_casualty_age).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Casualty_AgeGroup:O', title='Age group', sort='y') 
).properties(title=f"Casualty count percentage per Age group")

alt.Chart(...)

### Fatality Age Group

In [59]:
rc_fatal_age = rc[rc['Casualty_Severity'] == 'Fatality'].groupby('Casualty_AgeGroup')['Casualty_Count'].sum().reset_index(name='count') # .value_counts(normalize=True).reset_index(name='pct')
rc_fatal_age['pct'] = rc_fatal_age['count'] / rc_fatal_age['count'].sum()

alt.Chart(rc_fatal_age).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Casualty_AgeGroup:O', title='Age group', sort='y') 
).properties(title=f"Fatality count percentage per Age group")

alt.Chart(...)

The graph of Fatality age group percentages is similar to the above graph for Casualties overall. We see that the age group most likely to be in a car crash that results in a casualty or fatality is the 17 to 24 years of age group. This fact is supported by this document http://www.tmr.qld.gov.au/~/media/Safety/roadsafety/community%20road%20safety%20grants/resources/YoungDriversthefactsFinal.pdf which says that young drivers and riders are one of Queensland's most at risk road user groups. And that they are 60% more likely to be involved in a serious crash than mature adult drivers and riders. 

### Casualty/Fatality Gender

In [60]:
rc_fatal_gender = (rc[rc['Casualty_Severity'] == 'Fatality']
                   .groupby('Casualty_Gender')['Casualty_Count']
                   .sum()
                   .reset_index(name='count')
                  )
rc_fatal_gender['pct'] = rc_fatal_gender['count'] / rc_fatal_gender['count'].sum()

alt.Chart(rc_fatal_gender).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Casualty_Gender:O', title='Gender', sort='-x') 
).properties(title=f"Fatality count percentage per gender")

alt.Chart(...)

In [61]:
rc_fatal_gender['pct_change'] = rc_fatal_gender.pct.pct_change()
rc_fatal_gender

,Casualty_Gender,count,pct,pct_change
0,Female,1484,0.259123,NaN
1,Male,4232,0.738956,1.851752
2,Unknown,11,0.001921,-0.997401


This seems unusually high but the above table tells us that Males have an 85% more likely than Females to be in a fatal car accident.

In [62]:
rc_fatal_gender_age = (rc[rc['Casualty_Severity'] == 'Fatality']
                       .groupby(['Casualty_Gender', 'Casualty_AgeGroup'])
                       ['Casualty_Count'].sum()
                       .reset_index(name='count')
                      )

rc_fatal_gender_age['pct'] = rc_fatal_gender_age['count'] / rc_fatal_gender_age['count'].sum()

alt.Chart(rc_fatal_gender_age).mark_bar().encode(
    x=alt.X('Casualty_Gender', title='Gender', sort='y'),
    y=alt.Y('pct', axis=alt.Axis(format='%', title='Percentage'), stack=None),
    color=alt.Color('Casualty_Gender', title='Gender'),
    column='Casualty_AgeGroup'
).properties(title=f"Fatality count percentage per gender and age group")

alt.Chart(...)

We see that across all age groups, males are more likely to be in a fatal car crash than females.

### Fatal count per Road User Type

In [63]:
rc_fatal_user = (rc[rc['Casualty_Severity'] == 'Fatality']
                 .groupby('Casualty_RoadUserType')
                 ['Casualty_Count'].sum()
                 .reset_index(name='count')
                )

rc_fatal_user['pct'] = rc_fatal_user['count'] / rc_fatal_user['count'].sum()

fatal_user_chart = alt.Chart(rc_fatal_user).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Casualty_RoadUserType:O', title='Road User Type', sort='-x') 
)

fatal_user_text = fatal_user_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=alt.Text('pct', format='.2%')
)

(fatal_user_chart + fatal_user_text).properties(title=f"Fatality count percentage per Road User Type")

alt.LayerChart(...)

We see that pedestrians account for 11.80% of fatalities in car crashes

## Driver demographics

In [64]:
# driver demographics
# dg = pd.read_csv('b_driverdemographics.csv')
dg = pd.read_csv('https://www.data.qld.gov.au/dataset/f3e0ca94-2d7b-44ee-abef-d6b06e9b0729/resource/dd13a889-2a48-4b91-8c64-59f824ed3d2c/download/b_driverdemographics.csv')

In [65]:
dg.columns

Index(['Crash_Year', 'Crash_Police_Region', 'Crash_Severity',
       'Involving_Male_Driver', 'Involving_Female_Driver',
       'Involving_Young_Driver_16-24', 'Involving_Senior_Driver_60plus',
       'Involving_Provisional_Driver', 'Involving_Overseas_Licensed_Driver',
       'Involving_Unlicensed_Driver', 'Count_Crashes',
       'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised',
       'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury',
       'Count_Casualty_All'],
      dtype='object')

In [66]:
dg.head()

,Crash_Year,Crash_Police_Region,Crash_Severity,Involving_Male_Driver,Involving_Female_Driver,Involving_Young_Driver_16-24,Involving_Senior_Driver_60plus,Involving_Provisional_Driver,Involving_Overseas_Licensed_Driver,Involving_Unlicensed_Driver,Count_Crashes,Count_Casualty_Fatality,Count_Casualty_Hospitalised,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_All
0,2001,Brisbane,Fatal,No,No,No,No,No,No,No,1,1,0,0,0,1
1,2001,Brisbane,Fatal,No,Yes,No,No,No,No,No,4,4,1,0,0,5
2,2001,Brisbane,Fatal,No,Yes,No,Yes,No,No,No,2,2,0,0,0,2
3,2001,Brisbane,Fatal,No,Yes,No,Yes,No,No,Yes,1,1,0,0,0,1
4,2001,Brisbane,Fatal,No,Yes,No,Yes,Yes,No,No,1,1,1,0,0,2


### Driver demographic in most fatal crashes

In [67]:
fatal_dg = dg[dg['Crash_Severity'] == 'Fatal']

fatal_dg = fatal_dg.groupby(['Involving_Male_Driver', 'Involving_Female_Driver', 
                  'Involving_Young_Driver_16-24', 'Involving_Senior_Driver_60plus', 
                  'Involving_Provisional_Driver', 'Involving_Overseas_Licensed_Driver',
                  'Involving_Unlicensed_Driver'])['Count_Casualty_Fatality'].sum().reset_index(name='count')

fatal_dg.sort_values(by='count', ascending=False).head()

,Involving_Male_Driver,Involving_Female_Driver,Involving_Young_Driver_16-24,Involving_Senior_Driver_60plus,Involving_Provisional_Driver,Involving_Overseas_Licensed_Driver,Involving_Unlicensed_Driver,count
18,Yes,No,No,No,No,No,No,1666
23,Yes,No,No,Yes,No,No,No,706
30,Yes,No,Yes,No,Yes,No,No,491
27,Yes,No,Yes,No,No,No,No,428
1,No,Yes,No,No,No,No,No,357


We see that 1666 fatal crashes involved only Male Drivers, the next most group with the most fatal crashes was Male and/or Senior Drivers 60+ with 706 counts. Then it was Male and/or Provisional and Young Drivers (aged 16-24) with 491 fatalities.

TODO: compare fatality/crash rate for young drivers vs non-young drivers

## Seatbelt restraints and helmet use

In [68]:
# seatbelt restraints and helmet use
# rh = pd.read_csv('c_restraint_helmet_use.csv')
rh = pd.read_csv('https://www.data.qld.gov.au/dataset/f3e0ca94-2d7b-44ee-abef-d6b06e9b0729/resource/177dc50c-0cf7-46ba-8a69-99695aeaa46a/download/c_restraint_helmet_use.csv')

In [69]:
rh.columns

Index(['Crash_Year', 'Crash_PoliceRegion', 'Casualty_Severity',
       'Casualty_AgeGroup', 'Casualty_Gender', 'Casualty_Road_User_Type',
       'Casualty_Restraint_Helmet_Use', 'Casualty_Count'],
      dtype='object')

In [70]:
rh.head()

,Crash_Year,Crash_PoliceRegion,Casualty_Severity,Casualty_AgeGroup,Casualty_Gender,Casualty_Road_User_Type,Casualty_Restraint_Helmet_Use,Casualty_Count
0,2001,Brisbane,Fatality,0 to 16,Female,Vehicle Occupant,Restrained,1
1,2001,Brisbane,Fatality,17 to 24,Female,Vehicle Occupant,Restrained,1
2,2001,Brisbane,Fatality,25 to 29,Female,Vehicle Occupant,Restraint Use Not determined,1
3,2001,Brisbane,Fatality,30 to 39,Female,Vehicle Occupant,Restraint Use Not determined,1
4,2001,Brisbane,Fatality,30 to 39,Female,Vehicle Occupant,Unrestrained,1


### probability of fatality given restraint or helmet was used

In [71]:
fatal_rh = rh[rh['Casualty_Severity'] == 'Fatality']
fatal_rh = (fatal_rh.groupby('Casualty_Restraint_Helmet_Use')
            ['Casualty_Count'].sum()
            .reset_index(name='count')
           )

fatal_rh['pct'] = fatal_rh['count'] / fatal_rh['count'].sum()


fatal_rh_chart = alt.Chart(fatal_rh).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Casualty_Restraint_Helmet_Use:O', title='Restraint type', sort='-x') 
)

fatal_rh_text = fatal_rh_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=alt.Text('pct', format='.2%')
)

(fatal_rh_chart + fatal_rh_text).properties(title=f"Fatality pct for each restraint type")

alt.LayerChart(...)

38.57% of fatalities occur when driver is restrained, 14.30% of fatalities occur when unrestrained.

What proportion of unrestrained crashes result in a fatality?

In [72]:
unrestrained = rh[rh['Casualty_Restraint_Helmet_Use'] == 'Unrestrained']
unrestrained.head()

unrestrained = (unrestrained.groupby(['Casualty_Severity', 'Casualty_Restraint_Helmet_Use'])
                ['Casualty_Count'].sum()
                .reset_index(name='count')
               )

unrestrained['pct'] = unrestrained['count'] / unrestrained['count'].sum()


unrestrained_chart = alt.Chart(unrestrained).mark_bar().encode(
    x=alt.X('pct', axis=alt.Axis(format='%', title='Percentage')),
    y=alt.Y('Casualty_Severity:O', title='Restraint type', sort='-x') 
)

unrestrained_text = unrestrained_chart.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text=alt.Text('pct', format='.2%')
)

(unrestrained_chart + unrestrained_text).properties(title=f"Casualty Severity pct for Unrestrained Drivers/Passengers")

alt.LayerChart(...)

I expected there to be more fatalities for unrestrained drivers/passengers, but this doesn't really tell us anything unless we compare with the restrained drivers as well 

TODO: look at `Casualty_Road_User_Type`

## Vehicle types

In [73]:
# vehicle types
# vi = pd.read_csv('d_vehicle_involvement.csv')
vi = pd.read_csv('https://www.data.qld.gov.au/dataset/f3e0ca94-2d7b-44ee-abef-d6b06e9b0729/resource/f999155b-37f7-48aa-b5dd-644838130b0b/download/d_vehicle_involvement.csv')

In [74]:
vi.columns

Index(['Crash_Year', 'Crash_Police_Region', 'Crash_Severity',
       'Involving_Motorcycle_Moped', 'Involving_Truck', 'Involving_Bus',
       'Count_Crashes', 'Count_Casualty_Fatality',
       'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated',
       'Count_Casualty_MinorInjury', 'Count_Casualty_All'],
      dtype='object')

In [75]:
vi.head()

,Crash_Year,Crash_Police_Region,Crash_Severity,Involving_Motorcycle_Moped,Involving_Truck,Involving_Bus,Count_Crashes,Count_Casualty_Fatality,Count_Casualty_Hospitalised,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_All
0,2001,Brisbane,Fatal,No,No,No,36,38,18,1,0,57
1,2001,Brisbane,Fatal,No,No,Yes,1,1,1,0,0,2
2,2001,Brisbane,Fatal,No,Yes,No,1,1,0,1,2,4
3,2001,Brisbane,Fatal,Yes,No,No,5,5,0,2,0,7
4,2001,Brisbane,Fatal,Yes,Yes,No,1,1,0,0,0,1


### Vehicles most likely to be involved in a fatal car crash

In [76]:
fatal_vi = vi[vi['Crash_Severity'] == 'Fatal']

fatal_vi = (fatal_vi.groupby(['Involving_Motorcycle_Moped', 'Involving_Truck', 'Involving_Bus'])
            ['Count_Casualty_Fatality'].sum()
            .reset_index(name='count')
           )

fatal_vi.sort_values(by='count', ascending=False).head()

,Involving_Motorcycle_Moped,Involving_Truck,Involving_Bus,count
0,No,No,No,3645
4,Yes,No,No,960
2,No,Yes,No,931
1,No,No,Yes,85
6,Yes,Yes,No,81


Assuming "No" to each of the three columns means the accident only involve cars, then we can say that crashes involving only cars (no motorcycles, trucks or buses) have the most fatal crashes. And crashes involving motorcycles/mopeds (and cars) are the next group with the most fatal crashes. Followed by trucks and cars and then busses and cars. 

## Factors in road crashes

In [77]:
# factors in road crashes
# frc = pd.read_csv('e_alcohol_speed_fatigue_defect.csv')
frc = pd.read_csv('https://www.data.qld.gov.au/dataset/f3e0ca94-2d7b-44ee-abef-d6b06e9b0729/resource/18ee2911-992f-40ed-b6ae-e756859786e6/download/e_alcohol_speed_fatigue_defect.csv')

In [78]:
frc.columns

Index(['Crash_Year', 'Crash_Police_Region', 'Crash_Severity',
       'Involving_Drink_Driving', 'Involving_Driver_Speed',
       'Involving_Fatigued_Driver', 'Involving_Defective_Vehicle',
       'Count_Crashes', 'Count_Fatality', 'Count_Hospitalised',
       'Count_Medically_Treated', 'Count_Minor_Injury',
       'Count_All_Casualties'],
      dtype='object')

In [79]:
frc.head()

,Crash_Year,Crash_Police_Region,Crash_Severity,Involving_Drink_Driving,Involving_Driver_Speed,Involving_Fatigued_Driver,Involving_Defective_Vehicle,Count_Crashes,Count_Fatality,Count_Hospitalised,Count_Medically_Treated,Count_Minor_Injury,Count_All_Casualties
0,2001,Brisbane,Fatal,No,No,No,No,25,25,6,4,2,37
1,2001,Brisbane,Fatal,No,No,No,Yes,1,1,0,0,0,1
2,2001,Brisbane,Fatal,No,Yes,No,No,8,8,5,0,0,13
3,2001,Brisbane,Fatal,Yes,No,No,No,2,2,0,0,0,2
4,2001,Brisbane,Fatal,Yes,No,No,Yes,1,1,3,0,0,4


### Factors involved in fatal car crashes

In [80]:
fatal_frc = frc[frc['Crash_Severity'] == 'Fatal']

fatal_frc = (fatal_frc.groupby(['Involving_Drink_Driving', 'Involving_Driver_Speed', 
                                'Involving_Fatigued_Driver', 'Involving_Defective_Vehicle'])
            ['Count_Fatality'].sum()
            .reset_index(name='count')
           )

fatal_frc.sort_values(by='count', ascending=False).head()

,Involving_Drink_Driving,Involving_Driver_Speed,Involving_Fatigued_Driver,Involving_Defective_Vehicle,count
0,No,No,No,No,3387
4,No,Yes,No,No,688
7,Yes,No,No,No,671
11,Yes,Yes,No,No,513
2,No,No,Yes,No,268


We can see that most fatal car crashes do not involve any of drink driving, speeding, fatigued driver or defective vehicles. The next most common group with fatal crashes are crashes involving speeding drivers, followed by drunk drivers and drunk and speeding drivers. 